In [5]:
import sys
sys.path.insert(0, '../')
from src.envs.amod_env import Scenario, AMoD #, Star2Complete
from src.misc.utils import mat2str, dictsum
from mpc_baselines.MPC import MPC
import time
import os
import subprocess
from collections import defaultdict
import numpy as np
import gurobipy as gp
import json
    
# def solveRebFlow(env,res_path,desiredAcc,CPLEXPATH):
#     t = env.time
#     accRLTuple = [(n,int(round(desiredAcc[n]))) for n in desiredAcc]
#     accTuple = [(n,int(env.acc[n][t+1])) for n in env.acc]
#     edgeAttr = [(i,j,env.G.edges[i,j]['time']) for i,j in env.G.edges]
#     modPath = os.getcwd().replace('\\','/')+'/mod/'
#     OPTPath = os.getcwd().replace('\\','/')+'/MPC/'+res_path
#     if not os.path.exists(OPTPath):
#         os.makedirs(OPTPath)
#     datafile = OPTPath + f'data_{t}.dat'
#     resfile = OPTPath + f'res_{t}.dat'
#     with open(datafile,'w') as file:
#         file.write('path="'+resfile+'";\r\n')
#         file.write('edgeAttr='+mat2str(edgeAttr)+';\r\n')
#         file.write('accInitTuple='+mat2str(accTuple)+';\r\n')
#         file.write('accRLTuple='+mat2str(accRLTuple)+';\r\n')
#     modfile = modPath+'minRebDistRebOnly.mod'
#     if CPLEXPATH is None:
#         CPLEXPATH = "/opt/ibm/ILOG/CPLEX_Studio128/opl/bin/x86-64_linux/"
#     my_env = os.environ.copy()
#     my_env["LD_LIBRARY_PATH"] = CPLEXPATH
#     out_file =  OPTPath + f'out_{t}.dat'
#     with open(out_file,'w') as output_f:
#         subprocess.check_call([CPLEXPATH+"oplrun", modfile, datafile], stdout=output_f, env=my_env)
#     output_f.close()

#     # 3. collect results from file
#     flow = defaultdict(float)
#     with open(resfile,'r', encoding="utf8") as file:
#         for row in file:
#             item = row.strip().strip(';').split('=')
#             if item[0] == 'flow':
#                 values = item[1].strip(')]').strip('[(').split(')(')
#                 for v in values:
#                     if len(v) == 0:
#                         continue
#                     i,j,f = v.split(',')
#                     flow[int(i),int(j)] = float(f)
#     action = [flow[i,j] for i,j in env.edges]
#     return action

In [9]:
# MPC exact
number_nodes = 5
number_charge_levels = 5
chargers = []
for node in range(number_nodes):
    if node%2 ==1:
        chargers.append(node)
f = open('../data/scenario_test1x10.json')
data = json.load(f)
tripAttr = data["demand"]
reb_time = data["rebTime"]
total_acc = data["totalAcc"]

scenario = Scenario(N=number_nodes, charging_stations=chargers, number_charge_levels=number_charge_levels, sd=10, tripAttr = tripAttr, demand_ratio=1, reb_time=reb_time, total_acc = total_acc)
env = AMoD(scenario, beta=0.5)

# set Gurobi environment
gurobi_env = gp.Env(empty=True)
gurobi_env.setParam('WLSACCESSID', '8cad5801-28d8-4e2e-909e-3a7144c12eb5')
gurobi_env.setParam('WLSSECRET', 'a25b880b-8262-492f-a2e5-e36d6d78cc98')
gurobi_env.setParam('LICENSEID', 799876)
gurobi_env.setParam("OutputFlag",0)
gurobi_env.start()

mpc = MPC(env, gurobi_env)
opt_rew = []
obs = env.reset()
done = False
served = 0
rebcost = 0
opcost = 0
revenue = 0
t_0 = time.time()
time_list = []
while(not done):
    time_i_start = time.time()
    paxAction, rebAction = mpc.MPC_exact() 
    time_i_end = time.time()
    t_i = time_i_end - time_i_start
    time_list.append(t_i)
    
    obs, reward1, done, info = env.pax_step(paxAction)
    
    obs, reward2, done, info = env.reb_step(rebAction)
    opt_rew.append(reward1+reward2) 
    served += info['served_demand']
    rebcost += info['rebalancing_cost']
    opcost += info['operating_cost']
    revenue += info['revenue'] 
print(f'MPC: Reward {sum(opt_rew)}, Revenue {revenue},Served demand {served}, Rebalancing Cost {rebcost}, Operational Cost {opcost}, Avg.Time: {np.array(time_list).mean():.2f} +- {np.array(time_list).std():.2f}sec')
demand = sum([env.demand[i,j][t] for i,j in env.demand for t in range(0,60)])
print(demand, served/demand)

node_acc: (1, 0) 2
node_acc: (1, 1) 2
node_acc: (1, 2) 2
node_acc: (1, 3) 2
node_acc: (1, 4) 2
node_acc: (3, 0) 2
node_acc: (3, 1) 2
node_acc: (3, 2) 2
node_acc: (3, 3) 2
node_acc: (3, 4) 2
node_acc: (0, 3) 2
node_acc: (0, 2) 2
node_acc: (0, 1) 2
node_acc: (2, 2) 2
node_acc: (2, 1) 2
node_acc: (4, 2) 2
node_acc: (4, 1) 2
node_acc: (2, 3) 2
node_acc: (4, 3) 2
Changed value of parameter WLSACCESSID
Changed value of parameter WLSSECRET
Changed value of parameter LICENSEID


GurobiError: Web license service only available for container environments

In [35]:
# MPC-tri-level
scenario = Scenario(json_file="data/scenario_nyc_4.json", sd=10, demand_ratio=.5, json_hr=7,json_tstep=3)
env = AMoD(scenario, beta=0.5)

mpc = MPC(env, CPLEXPATH,T=10)
opt_rew = []
obs = env.reset()
done = False
served = 0
rebcost = 0
opcost = 0
revenue = 0
paxFlow = dict()
rebFlow = dict()
desiredAcc = dict()
rebAction = dict()
t_0 = time.time()
time_list = []
action_list = []
for _ in range(100):
    try:
        while(not done):
            t = env.time
            time_i_start = time.time()
            res_path = 'tri-level/'
            desiredAcc[t], paxFlow[t], rebFlow[t] = mpc.tri_level()
            action_list.append(list(desiredAcc[t].values()))
            time_i_end = time.time()
            t_i = time_i_end - time_i_start
            time_list.append(t_i)
            print(f"t_i: {t_i:.1f}sec")
            obs, reward1, done, info = env.pax_step(CPLEXPATH = CPLEXPATH, PATH = res_path)

            rebAction[t] = solveRebFlow(env,'reb_'+res_path,desiredAcc[t],CPLEXPATH)
            obs, reward2, done, info = env.reb_step(rebAction[t])
            opt_rew.append(reward1+reward2) 
            served += info['served_demand']
            rebcost += info['rebalancing_cost']
            opcost += info['operating_cost']
            revenue += info['revenue']
    except:
        continue
print(f'Downgraded MPC (three levels): Reward {sum(opt_rew)}, Revenue {revenue},Served demand {served}, Rebalancing Cost {rebcost}, Operational Cost {opcost}, Avg.Time: {np.array(time_list).mean():.1f} +- {np.array(time_list).std():.1f}sec')
demand = sum([env.demand[i,j][t] for i,j in env.demand for t in range(0,60)])
print(demand, served/demand)

t_i: 7.2sec
t_i: 6.0sec
t_i: 7.4sec
t_i: 6.5sec
t_i: 4.9sec
t_i: 7.1sec
t_i: 10.9sec
t_i: 11.8sec
t_i: 9.8sec
t_i: 9.3sec
t_i: 7.9sec
t_i: 14.7sec
t_i: 10.0sec
t_i: 9.2sec
t_i: 27.5sec
t_i: 10.3sec
t_i: 6.7sec
t_i: 8.6sec
t_i: 8.1sec
t_i: 7.0sec
t_i: 8.7sec
t_i: 7.8sec
t_i: 8.7sec
t_i: 9.4sec
t_i: 10.8sec
t_i: 14.1sec
t_i: 14.1sec
t_i: 57.7sec
t_i: 55.7sec
t_i: 32.7sec
t_i: 55.4sec
t_i: 59.3sec
t_i: 63.5sec
t_i: 87.8sec
t_i: 46.8sec
t_i: 88.2sec
t_i: 50.2sec
t_i: 87.5sec
t_i: 86.1sec
t_i: 93.2sec
t_i: 99.8sec
t_i: 102.1sec
t_i: 25.6sec
t_i: 28.1sec
t_i: 87.5sec
t_i: 66.8sec
t_i: 111.0sec
t_i: 125.1sec
t_i: 92.4sec
t_i: 40.4sec
t_i: 61.7sec
t_i: 27.1sec
t_i: 47.7sec
t_i: 42.7sec
t_i: 57.7sec
t_i: 75.4sec
t_i: 59.0sec
t_i: 96.3sec
t_i: 35.7sec
t_i: 82.8sec
Downgraded MPC (three levels): Reward 34521.80495003316, Revenue 78768.80495003318,Served demand 8861.0, Rebalancing Cost 4828.5, Operational Cost 44247.0, Avg.Time: 41.6 +- 34.7sec
9811 0.903169911324024


In [ ]:
# MPC exact
scenario = Scenario(json_file="data_/scenario_nyc_4.json", sd=10, demand_ratio=.5, json_hr=7,json_tstep=3)
env = AMoD(scenario, beta=0.5)

mpc = MPC(env, CPLEXPATH,T=10)
opt_rew = []
obs = env.reset()
done = False
served = 0
rebcost = 0
opcost = 0
revenue = 0
t_0 = time.time()
time_list = []
while(not done):
    time_i_start = time.time()
    paxAction, rebAction = mpc.MPC_exact() 
    time_i_end = time.time()
    t_i = time_i_end - time_i_start
#     print(f"t_i: {t_i:.1f}sec")
    time_list.append(t_i)
    
    obs, reward1, done, info = env.pax_step(paxAction)
    
    obs, reward2, done, info = env.reb_step(rebAction)
    opt_rew.append(reward1+reward2) 
    served += info['served_demand']
    rebcost += info['rebalancing_cost']
    opcost += info['operating_cost']
    revenue += info['revenue'] 
print(f'MPC: Reward {sum(opt_rew)}, Revenue {revenue},Served demand {served}, Rebalancing Cost {rebcost}, Operational Cost {opcost}, Avg.Time: {np.array(time_list).mean():.2f} +- {np.array(time_list).std():.2f}sec')
demand = sum([env.demand[i,j][t] for i,j in env.demand for t in range(0,60)])
print(demand, served/demand)

# three levels
# scenario = Star2Complete(sd = 10, grid_travel_time = 2, T = 16, star_demand = 7.5, complete_demand=1.5, 
#                         star_center = [9,10,13,14], beta=0.9, alpha = 0.5, ninit = 200, fix_price = True)

scenario = Scenario(json_file="data_/scenario_nyc_4.json", sd=10, demand_ratio=.5, json_hr=7,json_tstep=3)
env = AMoD(scenario, beta=0.5)

mpc = MPC(env, CPLEXPATH,T=10)
opt_rew = []
obs = env.reset()
done = False
served = 0
rebcost = 0
opcost = 0
revenue = 0
paxFlow = dict()
rebFlow = dict()
desiredAcc = dict()
rebAction = dict()
t_0 = time.time()
time_list = []
action_list = []
for _ in range(100):
    try:
        while(not done):
            t = env.time
            time_i_start = time.time()
            res_path = 'tri-level/'
            desiredAcc[t], paxFlow[t], rebFlow[t] = mpc.tri_level()
            action_list.append(list(desiredAcc[t].values()))
            time_i_end = time.time()
            t_i = time_i_end - time_i_start
            time_list.append(t_i)
            print(f"t_i: {t_i:.1f}sec")
            obs, reward1, done, info = env.pax_step(CPLEXPATH = CPLEXPATH, PATH = res_path)

            rebAction[t] = solveRebFlow(env,'reb_'+res_path,desiredAcc[t],CPLEXPATH)
            obs, reward2, done, info = env.reb_step(rebAction[t])
            opt_rew.append(reward1+reward2) 
            served += info['served_demand']
            rebcost += info['rebalancing_cost']
            opcost += info['operating_cost']
            revenue += info['revenue']
    except:
        continue
print(f'Downgraded MPC (three levels): Reward {sum(opt_rew)}, Revenue {revenue},Served demand {served}, Rebalancing Cost {rebcost}, Operational Cost {opcost}, Avg.Time: {np.array(time_list).mean():.1f} +- {np.array(time_list).std():.1f}sec')
demand = sum([env.demand[i,j][t] for i,j in env.demand for t in range(0,60)])
print(demand, served/demand)